In [16]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from pathlib import Path
import logging
import time, multiprocessing

import torch
import torch.nn as nn
import torch.utils.data as utils
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from scipy import stats

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import balanced_accuracy_score

import gc
gc.enable()

In [4]:
rng = np.random.RandomState(4)
datadir = Path('.')
logpath = Path('.')

In [5]:
# Hyperparameters
num_epochs = 100
num_classes = 3
batch_size = 200
learning_rate = 0.001
k = 10

In [6]:
def combine_and_split():
    ## Files downloaded from https://console.cloud.google.com/storage/quickdraw_dataset/full/numpy_bitmap
    f_fish = datadir / "full_numpy_bitmap_fish.npy"    
    f_flamingo = datadir / "full_numpy_bitmap_flamingo.npy"    
    f_firetruck = datadir / "full_numpy_bitmap_firetruck.npy"    

    fish = np.load(f_fish) 
    flamingo = np.load(f_flamingo) 
    firetruck = np.load(f_firetruck)

    X = np.vstack([fish,flamingo, firetruck])
    y = np.asarray(fish.shape[0]*[0] + flamingo.shape[0]*[1] + firetruck.shape[0]*[2])
    
    return(X,y)

def sort_keep_balance(X,y,block_lengths):
    # Sort data and labels into blocks that preserve class balance
    # X: data matrix
    # y : 1D class labels
    # block_lengths : Block sizes to sort X,y into that preserve class balance
    clss,counts = np.unique(y, return_counts=True)
    ratios = counts / sum(counts)
    class_idxs = [np.where(y==i)[0] for i in clss]

    sort_idxs = []
    
    prior_idxs = np.zeros(len(clss)).astype(int)
    for n in block_lengths:
        get_idxs = np.rint(n*ratios).astype(int)
        for idxs,prior_idx,next_idx in zip(class_idxs,prior_idxs,get_idxs):
            sort_idxs.append(idxs[prior_idx:next_idx])
        prior_idxs = get_idxs
        
    sort_idxs = np.hstack(sort_idxs)
    
    return((X[sort_idxs,:], y[sort_idxs]))

In [7]:
X,y = combine_and_split()

ns = np.linspace(2,np.log10(len(y)),7)
ns = np.power(10,ns).astype(int)

In [8]:
X,y = sort_keep_balance(X,y,ns)

In [13]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear((4)**2 * 64, 200)
        self.fc2 = nn.Linear(200, 3)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [19]:
def cnn_learn(Xtrain,ytrain,Xtest,ytest, model):
    ## Data loader
    batch_size = int(n)
    
    #Xn = X[0:n, ::]
    #yn = y[0:n]
    
    # Train Loaders
    tensor_x = torch.from_numpy(np.reshape(Xtrain,(Xtrain.shape[0],1,28,28))).float()
    tensor_y = torch.from_numpy(np.asanyarray(ytrain)).long()

    train_tensor = utils.TensorDataset(tensor_x,tensor_y) # create your datset

    train_loader = DataLoader(dataset=train_tensor, batch_size=batch_size, shuffle=True)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Train the model
    trainStartTime = time.time()
    
    total_step = len(train_loader)
    loss_list = []
    acc_list = []
    prior_loss = []
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            #images = images.float(); labels = labels.long()
            # Run the forward pass
            outputs = model(images.float())
            #print(outputs)
            #print(labels.long())
            loss = criterion(outputs, labels.long())
            loss_list.append(loss.item())

            # Backprop and perform Adam optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track the accuracy
            total = labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item()
            acc_list.append(correct / total)

            if ((i + 1) % 1 == 0) and ((epoch + 1) % 5 == 0):
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                              (correct / total) * 100))
                
        #if epoch > 10:
        #    if stats.sem(prior_loss[-5:]) < 0.001:
        #        break
        prior_loss.append(loss.item())
    
    trainEndTime = time.time()
    trainTime = trainEndTime - trainStartTime
    
    # Test Loaders
    tensor_x = torch.from_numpy(np.reshape(Xtest,(Xtest.shape[0],1,28,28))).float()
    tensor_y = torch.from_numpy(np.asarray(ytest)).long()

    test_tensor = utils.TensorDataset(tensor_x,tensor_y) # create your datset
    test_loader = DataLoader(dataset=test_tensor, batch_size=batch_size, shuffle=False)
    
    # Test the model
    testStartTime = time.time()
    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
        testEndTime = time.time()
        testTime = testEndTime - testStartTime
        print('Test Accuracy of the model on the test images: {} %'.format((correct / total) * 100))
        
        lhat = 1 - correct / total
        
        return((lhat,trainTime,testTime))

In [20]:
## Log output
write_path = logpath / 'hvbar_cnn_results_5runs.csv'

with open(write_path, 'w') as f:
    f.write("classifier,n,Lhat,trainTime,testTime,iterate\n")
    
logging.basicConfig(filename=logpath / 'mf_cnn_logging.log',
                        format='%(asctime)s:%(levelname)s:%(message)s',
                        level=logging.DEBUG
                        )
logging.info('NEW MF Quickdraw CNN RUN')

for n in tqdm(ns):
    logging.info(f'Run:n={n},clf=CNN')
    Xn = X[0:n, ::]
    yn = y[0:n]
    
    sss = StratifiedShuffleSplit(n_splits=k, test_size=1/k, random_state=0)
    sss.get_n_splits(Xn, yn)
    
    fit_times = []
    score_times = []
    test_scores = []
    
    for train_index, test_index in sss.split(Xn, yn):
        X_train, X_test = Xn[train_index], Xn[test_index]
        y_train, y_test = yn[train_index], yn[test_index]
    
        model = ConvNet()
        fit_t, score_t, test_score = cnn_learn(X_train, y_train,
                                               X_test, y_test, model)
        fit_times.append(fit_t)
        score_times.append(score_t)
        test_scores.append(test_score)

    ####("variable,Lhat,trainTime,testTime,iterate")
    with open(write_path, 'a') as f:
        for kfold, (trainTime, testTime, lhat) in enumerate(
                                         zip(fit_times,score_times,test_scores)):
            f.write(f"{'CNN'}, {int(n*(1-1/k))}, {1-lhat:2.9f}, {trainTime:2.9f}, {testTime:2.9f}, {kfold+1}\n")

    gc.collect()

  0%|          | 0/7 [00:00<?, ?it/s]

Epoch [5/100], Step [1/1], Loss: 5.4546, Accuracy: 57.78%
Epoch [10/100], Step [1/1], Loss: 0.2646, Accuracy: 92.22%
Epoch [15/100], Step [1/1], Loss: 0.1409, Accuracy: 94.44%
Epoch [20/100], Step [1/1], Loss: 0.1367, Accuracy: 95.56%
Epoch [25/100], Step [1/1], Loss: 0.0396, Accuracy: 98.89%
Epoch [30/100], Step [1/1], Loss: 0.0472, Accuracy: 97.78%
Epoch [35/100], Step [1/1], Loss: 0.0468, Accuracy: 97.78%
Epoch [40/100], Step [1/1], Loss: 0.0253, Accuracy: 98.89%
Epoch [45/100], Step [1/1], Loss: 0.0593, Accuracy: 98.89%
Epoch [50/100], Step [1/1], Loss: 0.0601, Accuracy: 98.89%
Epoch [55/100], Step [1/1], Loss: 0.0356, Accuracy: 98.89%
Epoch [60/100], Step [1/1], Loss: 0.0064, Accuracy: 100.00%
Epoch [65/100], Step [1/1], Loss: 0.0028, Accuracy: 100.00%
Epoch [70/100], Step [1/1], Loss: 0.0030, Accuracy: 100.00%
Epoch [75/100], Step [1/1], Loss: 0.0049, Accuracy: 100.00%
Epoch [80/100], Step [1/1], Loss: 0.0017, Accuracy: 100.00%
Epoch [85/100], Step [1/1], Loss: 0.0125, Accuracy: 

KeyboardInterrupt: 